In [12]:
import pandas as pd
import json
import requests
import geopy
from geopy import distance
from datetime import datetime

In [13]:
myurl = '/Users/yogevladani/study/year c/ניתוח בפייתון/מטלות/matala3/dests.txt'

In [14]:
destinations = [line.rstrip() for line in open(myurl)]
destinations
origion='tel aviv'
destinations

['Istanbul', 'Amsterdam', 'Valletta', 'Basel', 'Doha']

### Create list for the distance and duration and append the values for every destionation from the text file.

In [57]:
distances=[]
durations=[]

for i in range(0,len(destinations)):
    
    url="https://maps.googleapis.com/maps/api/distancematrix/json?origins=%s&destinations=%s&=imperial&key=%s" % (origion,destinations[i],api_key)

    try:
        response = requests.get(url)
        if not response.status_code == 200:
            print("HTTP error",response.status_code)
        else:
            try:
                response_data = response.json()
            except:
                print("Response not in valid JSON format")
    except:
        print("Something went wrong with requests.get")
    
    distances.append(response_data['rows'][0]['elements'][0]['distance']['value'])
    durations.append(response_data['rows'][0]['elements'][0]['duration']['value'])


### Convert the durations values to hours and minutes.
### There e-mach between the values and text becaues they dont show the seconds in the text and the values are in seconds.

In [98]:
def convert_to_hours_minutes(duration):
    # Convert duration to hours and minutes
    hours = duration // 3600
    minutes = (duration % 3600)//60
    return f"{hours} hours {minutes} minutes"

converted_durations=[]
for duration in durations:
    converted_durations.append(convert_to_hours_minutes(duration))
    


In [99]:
converted_durations

['21 hours 2 minutes',
 '48 hours 3 minutes',
 '50 hours 52 minutes',
 '44 hours 9 minutes',
 '22 hours 38 minutes']

### Create list for the lng and lat and append the values for every destionation from the text file.

In [100]:
lng=[]
lat=[]

for i in range (0,len(destinations)):
    url="https://maps.googleapis.com/maps/api/geocode/json?address=%s&key=%s"  % (destinations[i],api_key)
    try:
        response = requests.get(url)
        if not response.status_code == 200:
            print("HTTP error",response.status_code)
        else:
            try:
                response_data = response.json()
            except:
                print("Response not in valid JSON format")
    except:
        print("Something went wrong with requests.get")
    lat.append(response_data['results'][0]['geometry']['location']['lat'])
    lng.append(response_data['results'][0]['geometry']['location']['lng'])
    

In [101]:
print(lat,'\n',lng)

[41.0082376, 52.3675734, 35.8992375, 47.5595986, 25.2854473] 
 [28.9783589, 4.9041389, 14.5140996, 7.5885761, 51.53103979999999]


### Create the data

In [102]:
data={'Target':destinations,'Distance_km':distances,'Duration (hour+minutes)':converted_durations,'Longitude':lng,'latitude':lat}


In [103]:
data=pd.DataFrame(data)
data

,Target,Distance_km,Duration (hour+minutes),Longitude,latitude
0,Istanbul,1815227,21 hours 2 minutes,28.978359,41.008238
1,Amsterdam,4532541,48 hours 3 minutes,4.904139,52.367573
2,Valletta,3792882,50 hours 52 minutes,14.514100,35.899237
3,Basel,4118225,44 hours 9 minutes,7.588576,47.559599
4,Doha,2164477,22 hours 38 minutes,51.531040,25.285447


### Use nlargest(n) to get the 3 longest distances.

In [104]:
data.Distance_km.nlargest(3)

1    4532541
3    4118225
2    3792882
Name: Distance_km, dtype: int64

### The 3 farthest destinations.

In [105]:
data.loc[data['Distance_km'].nlargest(3).index, ['Target','Distance_km']]

,Target,Distance_km
1,Amsterdam,4532541
3,Basel,4118225
2,Valletta,3792882
